In [ ]:
#Importing everything

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
print("PyTorch:", torch.__version__)
print("GPU available:", torch.cuda.is_available())

from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.sdss import SDSS

print("Astropy + Astroquery loaded successfully ✅")


In [ ]:
#Query SDSS stellar spectra

from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

SDSS.clear_cache()
print("SDSS cache cleared.")

coords = SkyCoord(ra=2.023*u.degree, dec=14.84*u.degree, frame='icrs')
search_radius = 3 * u.arcmin

print("Querying SDSS for spectra...")
matches = SDSS.query_region(coords, radius=search_radius, spectro=True)

if matches is None:
    raise RuntimeError("SDSS returned None (server-side issue).")

print(f"Found {len(matches)} objects with spectra.")

matches = matches[:10]
spectra_hdus = SDSS.get_spectra(matches=matches)

print(f"Downloaded {len(spectra_hdus)} spectra.")


In [ ]:
#Extracting wavelength, flux, and metadata

data_list = []

for i, hdu_list in enumerate(spectra_hdus):
    try:
        header = hdu_list[0].header
        data = hdu_list[1].data

        #Converting log wavelength to wavelength (Angstrom)
        wavelength = 10 ** data['loglam']
        flux = data['flux']

        main_class = header.get('CLASS', 'Unknown')
        subclass = header.get('SUBCLASS', 'Unknown')
        if subclass is None or subclass.strip() == '':
            subclass = 'Unknown'

        data_list.append({
            'index': i,
            'class': main_class,
            'subclass': subclass,
            'wavelength': wavelength,
            'flux': flux,
            'num_points': len(wavelength)
        })

        print(f"Spectrum {i+1}: CLASS={main_class}, SUBCLASS={subclass}")

    except Exception as e:
        print(f"Skipped spectrum {i+1}: {e}")

#Creating DataFrame
df_stars = pd.DataFrame(data_list)

print("\nSummary table:")
df_stars[['index', 'class', 'subclass', 'num_points']]


In [ ]:
#Ploting the first spectrum (sanity check is important)

first = df_stars.iloc[0]

plt.figure(figsize=(12, 5))
plt.plot(first['wavelength'], first['flux'], lw=1)
plt.xlabel("Wavelength (Å)")
plt.ylabel("Flux")
plt.title("Raw SDSS Stellar Spectrum")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
#Define a common wavelength grid is a must

# Inspecting the wavelength coverages
min_waves = []
max_waves = []

for _, row in df_stars.iterrows():
    min_waves.append(row['wavelength'].min())
    max_waves.append(row['wavelength'].max())

global_min = np.max(min_waves)   # conservative overlap
global_max = np.min(max_waves)

print(f"Common wavelength range: {global_min:.1f} Å to {global_max:.1f} Å")

NUM_POINTS = 2000  # ML-friendly size
common_wavelength = np.linspace(global_min, global_max, NUM_POINTS)

print("Common wavelength grid shape:", common_wavelength.shape)


In [ ]:
#Interpolated all spectra onto common grids

from scipy.interpolate import interp1d

interpolated_fluxes = []

for i, row in df_stars.iterrows():
    wave = row['wavelength']
    flux = row['flux']


    interp_func = interp1d(
        wave,
        flux,
        kind='linear',
        bounds_error=False,
        fill_value="extrapolate"
    )

    new_flux = interp_func(common_wavelength)
    interpolated_fluxes.append(new_flux)

interpolated_fluxes = np.array(interpolated_fluxes)

print("Interpolated flux array shape:", interpolated_fluxes.shape)


In [ ]:
# Normalized each spectrum (z-score normalization)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

normalized_fluxes = []

for i in range(interpolated_fluxes.shape[0]):
    spectrum = interpolated_fluxes[i].reshape(-1, 1)
    norm_spectrum = scaler.fit_transform(spectrum).flatten()
    normalized_fluxes.append(norm_spectrum)

normalized_fluxes = np.array(normalized_fluxes)

print("Normalized flux array shape:", normalized_fluxes.shape)
print("Mean (first spectrum):", np.mean(normalized_fluxes[0]))
print("Std (first spectrum):", np.std(normalized_fluxes[0]))


In [ ]:
# Convert normalized spectra to PyTorch tensors

import torch

# Convert to float32 tensor
X = torch.tensor(normalized_fluxes, dtype=torch.float32)

print("Tensor shape:", X.shape)
print("Tensor dtype:", X.dtype)


In [ ]:
#Define a spectral autoencoder

import torch
import torch.nn as nn

class SpectralAutoencoder(nn.Module):
    def __init__(self, input_dim=2000, latent_dim=32):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 1024),
            nn.ReLU(),
            nn.Linear(1024, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z

# Instantiate model
model = SpectralAutoencoder(input_dim=X.shape[1], latent_dim=32)

print(model)


In [ ]:
#Train the autoencoder

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


dataset = TensorDataset(X)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 30

model.train()

for epoch in range(EPOCHS):
    epoch_loss = 0.0

    for batch in dataloader:
        batch_x = batch[0]

        optimizer.zero_grad()

        recon, _ = model(batch_x)
        loss = criterion(recon, batch_x)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Reconstruction Loss: {avg_loss:.6f}")


In [ ]:
#Extract latent embeddings

model.eval()

with torch.no_grad():
    _, latent_vectors = model(X)

latent_vectors = latent_vectors.numpy()

print("Latent embedding shape:", latent_vectors.shape)


In [ ]:
#Anomaly detection using Isolation Forest

from sklearn.ensemble import IsolationForest

# Initialize model
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.2,   # expect ~20% anomalies (adjustable)
    random_state=42
)

# Fit on latent space
iso_forest.fit(latent_vectors)

# Predict anomalies
anomaly_labels = iso_forest.predict(latent_vectors)
anomaly_scores = iso_forest.decision_function(latent_vectors)

# Convert labels: -1 = anomaly, +1 = normal
results = pd.DataFrame({
    'spectrum_index': df_stars['index'],
    'anomaly_label': anomaly_labels,
    'anomaly_score': anomaly_scores
})

print(results)


In [ ]:
#Visualize latent space with PCA

from sklearn.decomposition import PCA

# Reduce latent vectors to 2D
pca = PCA(n_components=2)
latent_2d = pca.fit_transform(latent_vectors)

# Plot
plt.figure(figsize=(6, 5))

for i in range(len(latent_2d)):
    if anomaly_labels[i] == -1:
        plt.scatter(latent_2d[i, 0], latent_2d[i, 1],
                    color='red', s=100, label='Anomaly' if i == 0 else "")
    else:
        plt.scatter(latent_2d[i, 0], latent_2d[i, 1],
                    color='blue', s=80, label='Normal' if i == 0 else "")

plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("Latent Space: Normal vs Anomalous Spectra")
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
#Plot anomalous vs normal spectra

# Identify indices
anomaly_idx = np.where(anomaly_labels == -1)[0][0]
normal_idx = np.where(anomaly_labels == 1)[0][0]

anomaly_spec = df_stars.iloc[anomaly_idx]
normal_spec = df_stars.iloc[normal_idx]

plt.figure(figsize=(12, 5))

plt.plot(anomaly_spec['wavelength'], anomaly_spec['flux'],
         label='Anomalous Spectrum', color='red', lw=1)

plt.plot(normal_spec['wavelength'], normal_spec['flux'],
         label='Normal Spectrum', color='blue', lw=1, alpha=0.7)

plt.xlabel("Wavelength (Å)")
plt.ylabel("Flux")
plt.title("Comparison: Anomalous vs Normal Stellar Spectrum")
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
#Reconstruction error per spectrum

model.eval()

reconstruction_errors = []

with torch.no_grad():
    reconstructions, _ = model(X)

    for i in range(X.shape[0]):
        original = X[i]
        reconstructed = reconstructions[i]

        # Mean Squared Error per spectrum
        mse = torch.mean((original - reconstructed) ** 2).item()
        reconstruction_errors.append(mse)

reconstruction_errors = np.array(reconstruction_errors)

# Create results table
recon_results = pd.DataFrame({
    'spectrum_index': df_stars['index'],
    'reconstruction_error': reconstruction_errors
})

# Sort by error (highest = most anomalous)
recon_results = recon_results.sort_values(
    by='reconstruction_error',
    ascending=False
)

print(recon_results)


In [ ]:
#Combine anomaly scores

# Normalize scores
iso_scores_norm = (anomaly_scores - anomaly_scores.min()) / (anomaly_scores.max() - anomaly_scores.min())
recon_scores_norm = (reconstruction_errors - reconstruction_errors.min()) / (reconstruction_errors.max() - reconstruction_errors.min())

combined_score = iso_scores_norm + recon_scores_norm

combined_results = pd.DataFrame({
    'spectrum_index': df_stars['index'],
    'iso_score': iso_scores_norm,
    'recon_score': recon_scores_norm,
    'combined_score': combined_score
}).sort_values(by='combined_score', ascending=False)

print(combined_results)


In [ ]:
# Real-time inference setup

model.eval()

def process_single_spectrum(spectrum_tensor):
    """
    Simulates real-time processing of ONE spectrum.
    Returns:
        reconstruction_error
        isolation_forest_score
        combined_score
    """
    with torch.no_grad():
        recon, latent = model(spectrum_tensor.unsqueeze(0))

        # Reconstruction error
        recon_error = torch.mean((spectrum_tensor - recon.squeeze()) ** 2).item()

        # Isolation Forest score (use latent vector)
        latent_np = latent.numpy()
        iso_score = iso_forest.decision_function(latent_np)[0]

        return recon_error, iso_score


In [ ]:
# Simulate real-time streaming detection

import time

print("Starting real-time stellar anomaly monitoring...\n")

for i in range(X.shape[0]):
    spectrum_tensor = X[i]

    recon_error, iso_score = process_single_spectrum(spectrum_tensor)

    status = "NORMAL"
    if iso_score < 0:   # Isolation Forest convention
        status = "⚠️ ANOMALY DETECTED"

    print(f"[Stream] Spectrum {i}")
    print(f"  Reconstruction error : {recon_error:.6f}")
    print(f"  Isolation score      : {iso_score:.6f}")
    print(f"  Status               : {status}")
    print("-" * 45)

    # Simulate delay (like live data)
    time.sleep(1)


In [ ]:
# Log real-time anomalies

alert_log = []

for i in range(X.shape[0]):
    spectrum_tensor = X[i]
    recon_error, iso_score = process_single_spectrum(spectrum_tensor)

    if iso_score < 0:  # anomaly condition
        alert_log.append({
            'spectrum_index': i,
            'reconstruction_error': recon_error,
            'isolation_score': iso_score,
            'alert': 'ANOMALY'
        })

alerts_df = pd.DataFrame(alert_log)

print("Real-time anomaly log:")
print(alerts_df)

# Save alerts (simulated alert archive)
alerts_df.to_csv("stellar_anomaly_alerts.csv", index=False)
print("\nAlerts saved to stellar_anomaly_alerts.csv")


In [ ]:
#Collect SDSS spectra safely without vstack

from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
import astropy.units as u

SDSS.clear_cache()

base_ra = 2.023
base_dec = 14.84

offsets = [
    (0.0, 0.0),
    (0.02, 0.0),
    (-0.02, 0.0),
    (0.0, 0.02),
    (0.0, -0.02)
]

all_spectra_hdus = []

print("Querying SDSS using multiple safe pointings...\n")

for dra, ddec in offsets:
    coord = SkyCoord(
        ra=(base_ra + dra) * u.degree,
        dec=(base_dec + ddec) * u.degree,
        frame='icrs'
    )

    matches = SDSS.query_region(
        coord,
        radius=3 * u.arcmin,
        spectro=True
    )

    if matches is None or len(matches) == 0:
        continue

    print(f"Found {len(matches)} spectra at offset ({dra}, {ddec})")

    # Limit per pointing (safety)
    matches = matches[:10]

    spectra = SDSS.get_spectra(matches=matches)

    for hdu in spectra:
        all_spectra_hdus.append(hdu)

print(f"\nTotal spectra collected: {len(all_spectra_hdus)}")


In [ ]:
# Preprocess the larger SDSS dataset

from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
import pandas as pd

# Extract wavelength & flux from collected spectra
data_list_big = []

for i, hdu_list in enumerate(all_spectra_hdus):
    try:
        header = hdu_list[0].header
        data = hdu_list[1].data

        wavelength = 10 ** data['loglam']
        flux = data['flux']

        data_list_big.append({
            'index': i,
            'wavelength': wavelength,
            'flux': flux
        })

    except Exception as e:
        print(f"Skipped spectrum {i}: {e}")

df_big = pd.DataFrame(data_list_big)
print(f"Spectra loaded for preprocessing: {len(df_big)}")

#Define common wavelength grid (reuse earlier logic)
min_waves = [row['wavelength'].min() for _, row in df_big.iterrows()]
max_waves = [row['wavelength'].max() for _, row in df_big.iterrows()]

global_min = np.max(min_waves)
global_max = np.min(max_waves)

NUM_POINTS = 2000
common_wavelength_big = np.linspace(global_min, global_max, NUM_POINTS)

print(f"Common wavelength range: {global_min:.1f} Å – {global_max:.1f} Å")

#  Interpolate spectra
interpolated_fluxes_big = []

for _, row in df_big.iterrows():
    interp_func = interp1d(
        row['wavelength'],
        row['flux'],
        bounds_error=False,
        fill_value="extrapolate"
    )
    interpolated_fluxes_big.append(interp_func(common_wavelength_big))

interpolated_fluxes_big = np.array(interpolated_fluxes_big)

print("Interpolated shape:", interpolated_fluxes_big.shape)

#  Normalize spectra
normalized_fluxes_big = []

scaler = StandardScaler()

for i in range(interpolated_fluxes_big.shape[0]):
    spec = interpolated_fluxes_big[i].reshape(-1, 1)
    norm_spec = scaler.fit_transform(spec).flatten()
    normalized_fluxes_big.append(norm_spec)

normalized_fluxes_big = np.array(normalized_fluxes_big)

print("Normalized shape:", normalized_fluxes_big.shape)

#Convert to PyTorch tensor
X_big = torch.tensor(normalized_fluxes_big, dtype=torch.float32)

print("Final tensor shape:", X_big.shape)


In [ ]:
# Retrain autoencoder on larger dataset

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Reinitialize model (fresh training)
model_big = SpectralAutoencoder(input_dim=2000, latent_dim=32)

# Dataset & loader
dataset_big = TensorDataset(X_big)
dataloader_big = DataLoader(dataset_big, batch_size=4, shuffle=True)

# Loss & optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model_big.parameters(), lr=1e-3)

# Training
EPOCHS = 50
model_big.train()

for epoch in range(EPOCHS):
    epoch_loss = 0.0

    for batch in dataloader_big:
        batch_x = batch[0]

        optimizer.zero_grad()
        recon, _ = model_big(batch_x)
        loss = criterion(recon, batch_x)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader_big)
    print(f"Epoch [{epoch+1}/{EPOCHS}] - Reconstruction Loss: {avg_loss:.6f}")


In [ ]:

from sklearn.ensemble import IsolationForest
import numpy as np
import pandas as pd

# Extract latent embeddings
model_big.eval()

with torch.no_grad():
    _, latent_big = model_big(X_big)

latent_big = latent_big.numpy()
print("Latent shape:", latent_big.shape)

#  Isolation Forest
iso_forest_big = IsolationForest(
    n_estimators=200,
    contamination=0.15,   # assume ~15% anomalies
    random_state=42
)

iso_forest_big.fit(latent_big)

anomaly_labels_big = iso_forest_big.predict(latent_big)
anomaly_scores_big = iso_forest_big.decision_function(latent_big)

#  Reconstruction error
reconstruction_errors_big = []

with torch.no_grad():
    recon_big, _ = model_big(X_big)

    for i in range(X_big.shape[0]):
        mse = torch.mean((X_big[i] - recon_big[i]) ** 2).item()
        reconstruction_errors_big.append(mse)

reconstruction_errors_big = np.array(reconstruction_errors_big)

#  Normalize & combine scores
iso_norm = (anomaly_scores_big - anomaly_scores_big.min()) / (anomaly_scores_big.max() - anomaly_scores_big.min())
recon_norm = (reconstruction_errors_big - reconstruction_errors_big.min()) / (reconstruction_errors_big.max() - reconstruction_errors_big.min())

combined_score_big = iso_norm + recon_norm

# Results table
results_big = pd.DataFrame({
    'spectrum_index': np.arange(len(X_big)),
    'iso_score': iso_norm,
    'recon_error': recon_norm,
    'combined_score': combined_score_big,
    'label': anomaly_labels_big
}).sort_values(by='combined_score', ascending=False)

print(results_big)


In [ ]:


model_big.eval()

for param in model_big.parameters():
    param.requires_grad = False

print("Model frozen for inference.")


In [ ]:


def streaming_inference(
    spectrum_tensor,
    model,
    iso_model
):
    """
    Process ONE incoming spectrum in real time.
    Returns a dictionary with anomaly information.
    """

    with torch.no_grad():
        # Forward pass
        recon, latent = model(spectrum_tensor.unsqueeze(0))

        # Reconstruction error
        recon_error = torch.mean((spectrum_tensor - recon.squeeze()) ** 2).item()

        # Isolation Forest score
        latent_np = latent.numpy()
        iso_score = iso_model.decision_function(latent_np)[0]
        iso_label = iso_model.predict(latent_np)[0]

        # Combined score (simple fusion)
        combined_score = recon_error - iso_score  # higher = more anomalous

    return {
        "reconstruction_error": recon_error,
        "isolation_score": iso_score,
        "isolation_label": iso_label,
        "combined_score": combined_score
    }

print("Streaming inference engine ready.")


In [ ]:


import time

candidate_log = []

print("Starting continuous stellar spectrum stream...\n")

for i in range(len(X_big)):
    spectrum = X_big[i]

    result = streaming_inference(
        spectrum_tensor=spectrum,
        model=model_big,
        iso_model=iso_forest_big
    )

    print(f"[STREAM] Spectrum {i}")
    print(f"  Reconstruction error : {result['reconstruction_error']:.6f}")
    print(f"  Isolation score      : {result['isolation_score']:.6f}")
    print(f"  Combined score       : {result['combined_score']:.6f}")

    # Decide anomaly
    if result["isolation_label"] == -1:
        print("  ⚠️  FLAGGED AS ANOMALY\n")

        candidate_log.append({
            "spectrum_index": i,
            "reconstruction_error": result["reconstruction_error"],
            "isolation_score": result["isolation_score"],
            "combined_score": result["combined_score"]
        })
    else:
        print("  Status: Normal\n")

    time.sleep(0.5)  # simulate streaming delay

print("Streaming complete.")
print(f"Total candidates flagged: {len(candidate_log)}")


In [ ]:
#Long-running batch ingestion simulation

import time

long_run_candidates = []

NUM_CYCLES = 5   # simulate 5 observing cycles

print("Starting long-running observation simulation...\n")

for cycle in range(NUM_CYCLES):
    print(f"=== Observation cycle {cycle+1}/{NUM_CYCLES} ===")

    for i in range(len(X_big)):
        spectrum = X_big[i]

        result = streaming_inference(
            spectrum_tensor=spectrum,
            model=model_big,
            iso_model=iso_forest_big
        )

        if result["isolation_label"] == -1:
            long_run_candidates.append({
                "cycle": cycle,
                "spectrum_index": i,
                "reconstruction_error": result["reconstruction_error"],
                "isolation_score": result["isolation_score"],
                "combined_score": result["combined_score"]
            })

    time.sleep(0.5)

print("\nLong-running ingestion complete.")
print(f"Total anomaly events detected: {len(long_run_candidates)}")


In [ ]:
# Identify stable anomaly candidates

import pandas as pd

# Convert to DataFrame
long_run_df = pd.DataFrame(long_run_candidates)

# Count how many times each spectrum was flagged
candidate_summary = (
    long_run_df
    .groupby("spectrum_index")
    .size()
    .reset_index(name="times_flagged")
    .sort_values(by="times_flagged", ascending=False)
)

print("Stable anomaly candidates:")
print(candidate_summary)


In [ ]:
# Inspect stable anomaly spectra (raw view)

candidate_indices = [3, 12]

plt.figure(figsize=(14, 6))

for idx in candidate_indices:
    row = df_big.iloc[idx]
    plt.plot(
        row['wavelength'],
        row['flux'],
        label=f"Candidate spectrum {idx}",
        lw=1
    )

plt.xlabel("Wavelength (Å)")
plt.ylabel("Flux")
plt.title("Stable Anomaly Candidate Spectra")
plt.legend()
plt.grid(alpha=0.3)
plt.show()
